## 导包

In [1]:
# import packages
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
import collections
warnings.filterwarnings('ignore')

In [2]:
data_path='/kaggle/input/news-recommendation-dataset/'
save_path='/kaggle/working/'

## df节省内存函数

In [3]:
# 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

## 读取采样或全量数据

In [4]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        从完整的训练集中采样一部份数据，用于调试和开发过程中快速迭代和测试
        data_path: 原数据的存储路径
        sample_nums: 采样数量
    """
    # 读取训练点击日志
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    # 获取所有独立用户ID
    all_user_ids = all_click.user_id.unique()
    
    # 随机采样用户
    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    # 筛选用户的点击数据
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    # 去重
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果应该讲测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(data_path='/kaggle/input/news-recommendation-dataset/', offline=True):
    """
        根据是否处理离线状态，读取并准备完整的点击数据集
        :param data_path：存储原始数据文件的路径
        :param offline：指示是否处于离线模式。在离线模式下只使用训练集      
    """
    if offline: # 离线模式--只读取训练集的点击数据
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else: # 在线模式--读取训练集和测试集的点击数据
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = pd.concat([tst_click,trn_click])
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [5]:
# 全量训练集
all_click_df = get_all_click_df(data_path, offline=False)

## 获取 用户-文章-点击时间 字典

In [6]:
# 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')[['click_article_id', 'click_timestamp']].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

## 获取点击最多的topk个文件

In [7]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## itemcf的物品相似度计算

In [8]:
def itemcf_sim(df):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [9]:
i2i_sim = itemcf_sim(all_click_df)

100%|██████████| 250000/250000 [00:33<00:00, 7378.14it/s] 


## itemcf的文章推荐

In [10]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 {item1:score1, item2: score2...}
        注意: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    # 获取用户的历史点击文章列表
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}
    
    # 初始化召回文章的得分字典（用于存储召回的文章及其相应的得分）
    item_rank = {}
    # 基于物品相似性进行召回
    for loc, (i, click_time) in enumerate(user_hist_items):
        # user_hist_items: {user1:[(item1:time1),(item2:time2),...],user2:...}
        # enumerate函数用于获取用户历史点击的索引，和每篇文章的id以及点击时间
        # i2i_sim是物品相似性矩阵，抱歉了当前文章i与其他所有文章的相似度得分
        # sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)：将这些键值对按照相似度得分（每个元组的第二个元素，即x[1]，也就是相似度得分）降序排序
        # 从排序后的列表中取出相似度最高的前sim_item_topk个文章
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            # wij表示文章i与文章j的相似度得分
            # 过滤用户已点击的文章
            if j in user_hist_items_:
                continue
            
            # 用户没点击过的文章
            item_rank.setdefault(j, 0) # 添加到item_rank字典中
            item_rank[j] +=  wij # 累加他们的相似度得分
    
    # 使用热门文章进行补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    # 将item_rank中的文章按照得分降序排序
    # 截取排序后的列表中的前recall_item_num篇文章作为最终召回结果
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

## 给每个用户根据物品的协同过滤推荐文章

In [11]:
# 定义
user_recall_items_dict = collections.defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章数量
recall_item_num = 10

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)

100%|██████████| 250000/250000 [43:04<00:00, 96.74it/s] 


## 召回字典转换成df

In [12]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])

100%|██████████| 250000/250000 [00:05<00:00, 49593.77it/s]


## 生成提交文件

In [13]:
def submit(recall_df,topk=5,model_name=None):
    recall_df=recall_df.sort_values(by=['user_id','pred_score'])
    recall_df['rank']=recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False,method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp=recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min()>=topk
    
    del recall_df['pred_score']
    submit=recall_df[recall_df['rank']<=topk].set_index(['user_id','rank']).unstack(-1).reset_index()
    
    submit.columns=[int(col) if isinstance(col,int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit=submit.rename(columns={'':'user_id',1:'article_1',2:'article_2',3:'article_3',4:'article_4',5:'article_5'})
    
    save_name=save_path+model_name+'_'+datetime.today().strftime('%m-%d')+'.csv'

    submit.to_csv(save_name,index=False,header=True)

In [14]:
# 获取测试集
tst_click=pd.read_csv(data_path+'testA_click_log.csv')
tst_users=tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall=recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall,topk=5,model_name='itemcf_baseline')